In this project, we are building a multi-agent system using the crewai library to perform market research on emerging trends in the restaurant industry. The focus areas are digital menus, AI in restaurants, and pricing strategies. The system consists of two main agents:

* Trend Research Agent: Gathers relevant data on specified topics.
* Market Researcher Agent: Synthesizes the gathered information into a coherent report.

# Setup

First, we install the necessary libraries and retrieve the API keys required for accessing OpenAI's language models and performing web searches.

In [1]:
# Install the 'crewai' library and its tools, along with 'openai' and 'serper' for search capabilities
!pip install crewai
!pip install 'crewai[tools]'
!pip install openai
!pip install serper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Retrieve the API keys securely from Google Colab's user data
from google.colab import userdata
openai_api_key = userdata.get('genai_course')  # OpenAI API key for language models
serper_api_key = userdata.get('serper_api')    # Serper API key for web search capabilities

We are working in a Google Colab environment and need to install the required packages. The crewai library allows us to create and manage AI agents. The serper library provides web search functionality. API keys are securely retrieved using userdata.get() to ensure sensitive information is not exposed.

Next, we import the essential libraries for agent creation and set the API keys as environment variables.

In [2]:
# Import essential libraries for agent creation and web tools
import os
from crewai import Agent, Task, Crew, Process          # Core components for creating agents and tasks
from crewai_tools import SerperDevTool, ScrapeWebsiteTool  # Tools for web searching and scraping
from IPython.display import display, Markdown          # For displaying outputs in a Jupyter notebook
from langchain_openai import ChatOpenAI

In [ ]:
# Set the API keys as environment variables for accessibility
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

By setting the API keys as environment variables, we ensure that they are accessible to any part of the program that requires them, such as the language model interface and web tools.

# Tools

We define the tools that our agents will use to perform web searches and scrape website content.

In [ ]:
# Initialize the tool for performing web searches using Serper API
search_tool = SerperDevTool()

# Initialize the tool for scraping website content
scrape_tool = ScrapeWebsiteTool()

The SerperDevTool allows agents to perform web searches, while the ScrapeWebsiteTool enables them to extract content from websites. These tools expand the agents' capabilities beyond text generation to include data retrieval from the internet.

# Research AI Agent + Task

We define the Trend Research Agent responsible for gathering relevant data.

In [ ]:
# Define the Research AI Agent with its role, goal, and backstory
research_agent = Agent(
    role="Trend Research Agent",
    goal="Retrieve relevant data on digital menus, AI in restaurants, and pricing strategies",
    backstory="You are responsible for gathering relevant information on key trends in the restaurant industry for 2024 and beyond",
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4o model
    tools=[search_tool, scrape_tool],  # Assigning the search and scrape tools
    verbose=True
)

The agent's role, goal, and backstory provide context that guides its behavior. By specifying the language model and tools, we define how the agent will perform its tasks. The temperature parameter in the language model influences the creativity of the output.

Tasks are assigned to agents, meaning that one agent can have multiple tasks.

In the crewai framework, agents can handle multiple tasks. This allows for efficient use of resources and enables agents to perform related tasks without the need for additional agents.

We create two tasks for the Trend Research Agent:

1. Trend Task: Research emerging trends in digital menus.
2. Pricing Task: Research new pricing strategies for restaurants.

In [ ]:
# Create a task for researching trends in digital menus
trend_task = Task(
    description="Research emerging trends in 2024 and 2025 on digital menus",
    expected_output="All the information organized with key trends and insights for Bitte's MVP and future plans",
    agent=research_agent  # Assigning the task to the research agent
)

In [ ]:
# Create a task for researching pricing strategies
pricing_task = Task(
    description="Research new pricing strategies for restaurants for takeaway and delivery",
    expected_output="All the retrieved information organized with key trends and insights for Bitte's MVP and future plans",
    agent=research_agent  # Assigning the task to the same agent
)

By assigning both tasks to the research_agent, we leverage its capabilities to gather all necessary data. The expected_output field defines what we anticipate receiving from the agent after task completion.

# Writer Agent

We define the Market Researcher Agent responsible for synthesizing the data into a coherent report.

In [ ]:
# Define the writer agent to generate reports
writer_agent = Agent(
    role="Market Researcher that provides reports",
    goal="Generate reports based on search results",
    backstory="You are an experienced Market Researcher that synthesizes search results into a coherent report",
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
    verbose=True
)

The writer_agent is designed to process the data collected by the research_agent and compile it into a structured report. The agent's role and backstory help guide its behavior during the task.

We define the writing task for the writer_agent, using the outputs of the previous tasks as context.



In [ ]:
# Define the writing task
writing_task = Task(
    description="Generate a report based on the retrieved search data.",
    expected_output="A detailed and coherent report in Markdown on the trends in menu design, AI in restaurants, and pricing strategies. The report must start with an executive summary, followed by the content per topic, and conclude with actionable recommendations",
    agent=writer_agent,
    context=[trend_task, pricing_task]  # Using previous tasks as context
)

The context parameter allows the writer_agent to access the outputs from the trend_task and pricing_task. This is essential for the agent to incorporate the research findings into the final report.

# Assemble the team of AI Agents

We create a crew to manage the agents and tasks, specifying that tasks should be executed sequentially.

In [ ]:
# Create the AI team for market research
crew = Crew(
    agents=[research_agent, writer_agent],  # List of agents involved
    tasks=[trend_task, pricing_task, writing_task],  # List of tasks to execute
    verbose=True,
    process=Process.sequential  # Ensuring tasks are processed in order
)
result = crew.kickoff()  # Start the task execution

# Agent: Trend Research Agent
## Task: Research emerging trends in 2024 and 2025 on digital menus


# Agent: Trend Research Agent
## Thought: Thought: I need to gather information on emerging trends for digital menus in 2024 and 2025. I'll start by searching for relevant information on this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"emerging trends digital menus 2024 2025\"}"
## Tool Output: 

Search results: Title: Future Trends in Digital Menu Boards for 2024-2025 - DotSignage
Link: https://www.dotsignage.com/blog/future-trends-in-digital-menu-boards/
Snippet: Future trends of Digital Menu Boards in 2024 – Real time dynamic menus, AI integrations and gamification of digital menu boards.
---
Title: Digital Menu Board Trends for 2024 | Eye Catch - EyeCatch Networks
Link: https://www.eyecatch.co/top-5-digital-menu-board-trends-for-2024
Snippet: Top 5 Digital Menu Board Trends for 2024 · 1. Interactive Touchscreen Menu Boards · 2. Dynamic Content and 

The Crew object orchestrates the agents and tasks. By setting process=Process.sequential, we ensure that tasks are completed in order, which is important when tasks depend on the outputs of previous tasks.

**ReACT prompt engineering technique**
ReACT (Reasoning and Acting) is a prompt engineering technique where the AI agent is designed to reason through tasks and decide on actions. It involves generating reasoning traces and actions that guide the agent towards the desired outcome. This technique improves the agent's ability to handle complex tasks by combining reasoning with action.

Finally, we extract and display the outputs from each task in a readable format using Markdown. We iterate over the task outputs and use Markdown to format the display. This helps in presenting the results in a structured and readable manner within a Jupyter notebook.

In [ ]:
# Extract and display each agent's output in Markdown
for idx, task_output in enumerate(result.tasks_output):
  display(Markdown(f"### agent {idx +1}: task_output.agent]\n{task_output.raw}"))

### agent 1: task_output.agent]
Future Trends in Digital Menu Boards for 2024-2025

Digital signage is transforming the restaurant industry, and digital menu boards are at the forefront of this change. These boards offer numerous advantages, such as faster updates, enhanced visuals, and more interactive experiences, thus improving customer engagement and operational efficiency.

1. **AI & Machine Learning**: AI and machine learning are enhancing digital displays, providing personalized customer experiences. They analyze customer data to suggest personalized meals, considering dietary restrictions and trends. This customization boosts sales and customer satisfaction.

2. **Interactive Nutritional Information**: Digital menus are becoming interactive, allowing customers to access nutritional information easily. By tapping on a dish, customers can view its nutritional content, leading to more informed and healthier dining choices. This feature is still developing but holds great potential for transparency and customer engagement.

3. **Real-time Dynamic Menus**: These features allow for instant updates on specials, sold-out items, and price changes, ensuring customers always have the latest menu information. Dynamic content like videos and countdowns can enhance the dining experience and encourage upselling.

4. **Order Directly from Digital Menus**: Integration with mobile apps enables customers to place orders directly from digital menus, customize meals, and make payments. This not only streamlines the dining experience but also enhances operational efficiency and reduces errors.

5. **Gamification**: Incorporating gamification into menu boards can increase customer engagement. Features like interactive challenges, spin-the-wheel promotions, and social media integration can enhance customer loyalty and brand visibility.

6. **Multi-Sensory Integration**: Future digital menus will offer multi-sensory experiences, engaging sight, sound, and even smell to create a more immersive dining experience. This trend is in its early stages but promises to transform customer engagement significantly.

7. **Augmented Reality (AR) Overlays**: AR technology is set to revolutionize digital menus by allowing customers to view dishes in 3D through their smartphones. This interactive element enhances visual appeal and helps customers make informed decisions.

Digital menus are crucial for enhancing customer experiences and operational efficiency in the restaurant industry. As technology evolves, these features will continue to develop, offering even more innovative ways to engage customers and optimize restaurant operations.

### agent 2: task_output.agent]
The State of Dynamic Pricing for Restaurants in 2024, and How to Navigate

Industry Insights
MAY 02, 2024

California's recent decision to increase the minimum wage for fast food workers to $20 has sparked a critical conversation within the restaurant industry about the concept of price uplifting or dynamic pricing. This approach, which adjusts prices based on various factors like demand, time of day, or cost pressures, is common in sectors such as ride-sharing and airlines but has seen mixed reactions when applied to dining establishments.

Dynamic pricing across other industries looks like this:
- Airlines charge more for passenger tickets during popular times of the year.
- Amazon changes product prices every 10 minutes, boosting profits by 25%.
- Uber implements surge pricing in high-demand areas.
- Airbnb adjusts host prices based on demand.
- Disney World varies ticket prices with seasons.
- Grocery stores in Norway reduced food waste by 40% using dynamic pricing since 2012.

Dynamic pricing in restaurants is a strategy where menu prices are adjusted in real-time based on triggers like increased demand during peak hours, higher input costs, or weather changes. This model helps manage profitability and moderates customer flow. Restaurants often mark up prices on third-party delivery apps to offset high fees, averaging a 24% increase per meal. Despite the costs, 63% of consumers order from delivery apps for speed and convenience, while 38% say it's their only option for certain restaurants.

New wage laws prompted notable chains like Chipotle, Wendy's, and Taco Bell to adjust their pricing strategies, leading to consumer backlash. Dynamic pricing can lead to a sensitivity to perceived price fairness in food services, contrasting with more acceptance in other industries.

Implementing dynamic pricing involves:
- Transparency: Clear communication about price changes helps mitigate negative reactions.
- Customer Segmentation: Tailor pricing strategies effectively by offering off-peak discounts or loyalty perks.
- Market Conditions: Balance profitability with customer satisfaction during high demand or cost fluctuations.
- Technological Integration: Solutions like Lunchbox enable seamless implementation of dynamic pricing. These systems can adjust prices by delivery partner, location, or market conditions, allowing nuanced pricing adjustments.

Lunchbox's Order Aggregation solution allows customizable dynamic pricing strategies for specific delivery partners and locations. Technology supports effective price management and offers data analytics to monitor pricing strategies' impact on sales and customer satisfaction. With Order Aggregation, restaurants can adjust pricing by:
- Dollar or percentage per item or menu category
- Set percentages per delivery partner
- Adjust item prices without changing modifiers
- Pass third-party discounts back to the POS for simplified accounting

Conclusion: While dynamic pricing is powerful, its success depends on thoughtful implementation and management. As the industry adapts to new economic realities, restaurants must weigh strategies carefully, ensuring they meet business goals while respecting consumer expectations.

### agent 3: task_output.agent]
# Restaurant Industry Trends Report: 2024-2025

## Executive Summary
The restaurant industry is undergoing a significant transformation driven by advancements in technology and evolving consumer expectations. This report explores three critical areas shaping the future: trends in menu design, the integration of AI in restaurants, and innovative pricing strategies. We analyze how these trends contribute to operational efficiency, customer satisfaction, and business growth. The report concludes with actionable recommendations for restaurant operators to navigate these changes effectively.

## Digital Menu Trends

### 1. AI & Machine Learning
Digital menu boards are increasingly incorporating AI and machine learning to personalize customer experiences. By analyzing customer data, AI can suggest meals tailored to individual preferences, dietary restrictions, and emerging trends. This level of customization not only enhances the dining experience but also boosts sales and customer loyalty.

### 2. Interactive Nutritional Information
The push for transparency in dining choices has led to the development of interactive digital menus. These allow customers to access detailed nutritional information by interacting directly with the menu items. This feature promotes healthier dining decisions and increases customer satisfaction by providing clear, accessible information.

### 3. Real-time Dynamic Menus
Restaurants are leveraging technology to offer real-time menu updates. Digital boards can reflect changes in specials, sold-out items, and price adjustments instantly. This ensures that customers always have the most current information, which enhances their dining experience and supports upselling through dynamic content like videos and promotional countdowns.

### 4. Direct Ordering and Payment Integration
Digital menus are being integrated with mobile applications, allowing customers to place orders and make payments directly, streamlining the entire dining process. This reduces the likelihood of errors and enhances operational efficiency, benefiting both the restaurant and the customer.

### 5. Gamification
Incorporating elements of gamification into digital menus is a growing trend. By engaging customers through interactive challenges, promotions, and social media integration, restaurants can enhance customer loyalty and increase brand visibility.

### 6. Multi-Sensory Experiences
The future of digital menus includes multi-sensory integration, where experiences are designed to engage sight, sound, and possibly even smell. While still in its early stages, this trend has the potential to significantly elevate customer engagement and create memorable dining experiences.

### 7. Augmented Reality Overlays
AR technology is revolutionizing the way customers interact with menus, allowing them to visualize dishes in 3D. This enhances the visual appeal of menu items and helps customers make more informed decisions, thereby improving overall satisfaction.

## AI in Restaurants

The integration of AI into restaurant operations is becoming more pronounced, with significant impacts on efficiency and personalization. AI systems can streamline operations by predicting busy times, managing inventory, and even automating customer service interactions. The ability to gather and analyze vast amounts of data allows for more precise marketing and personalized customer interactions, driving both efficiency and customer satisfaction.

## Dynamic Pricing Strategies

### Overview
Dynamic pricing in the restaurant industry involves adjusting prices based on demand, time of day, and other external factors. While this strategy is common in industries like airlines and ride-sharing, its application in restaurants is still evolving.

### Industry Examples
Other sectors successfully using dynamic pricing include airlines, Amazon, Uber, Airbnb, and Disney World, each adjusting prices based on demand or time. For restaurants, this means adapting pricing in response to factors like peak dining times, cost fluctuations, or weather conditions.

### Implementation in Restaurants
Dynamic pricing can improve profitability and manage customer flow, but it also requires careful implementation:
- **Transparency**: Clear communication about price changes is vital to avoid negative customer reactions.
- **Customer Segmentation**: Offering discounts during off-peak hours or providing loyalty perks can tailor the pricing experience.
- **Technological Solutions**: Tools like Lunchbox enable seamless dynamic pricing, allowing adjustments by location, delivery partner, or market conditions.

## Conclusion and Recommendations

### Recommendations
1. **Invest in Technology**: Embrace digital menu boards and AI solutions to enhance customer engagement and streamline operations.
2. **Communicate Clearly**: Ensure transparency in pricing strategies to maintain consumer trust and satisfaction.
3. **Leverage Data**: Use AI and data analytics to inform menu design and pricing decisions, tailoring offerings to customer preferences.
4. **Innovate Customer Experience**: Explore gamification and AR overlays to create memorable dining experiences that differentiate your brand.
5. **Balance Profitability and Fairness**: Implement dynamic pricing thoughtfully, keeping consumer expectations and market conditions in mind.

By adopting these strategies, restaurant operators can effectively navigate the evolving landscape, driving growth and maintaining a competitive edge in the industry.